In [ ]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import numpy as np
import dask
from dask.dot import dot_graph
from elfi.core import *
from elfi.distributions import *
from elfi.methods import Rejection, BOLFI
from elfi.examples.ma2 import MA2, autocov, distance
from elfi.visualization import draw_model
from distributed import Client
from functools import partial

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

### Generate some toy data

In [ ]:
n = 100
t1_0 = 0.2
t2_0 = 0.6

# Set up observed data y
latents = np.random.RandomState(12345).randn(n+2)
y = MA2(n, 1, t1_0, t2_0, latents=latents)

# Plot
plt.figure(figsize=(11, 6))
plt.plot(np.arange(0,n),y[0,:])
plt.scatter(np.arange(-2,n), latents);

### Setup the simultator and summary functions

In [ ]:
# Set up the MA 2 simulator
# Fix n to the same value as in the observed data
simulator = partial(MA2, n)

# Set up autocovariance summaries for lags 1 and 2
ac1 = partial(autocov, 1)
ac2 = partial(autocov, 2)

### Build the model

In [ ]:
# Specify the graphical model
t1 = Prior('t1', 'uniform', 0, 1)
t2 = Prior('t2', 'uniform', 0, 1)
Y = Simulator('MA2', simulator, t1, t2, observed=y)
S1 = Summary('S1', ac1, Y)
S2 = Summary('S2', ac2, Y)
d = Discrepancy('d', distance, S1, S2)

In [ ]:
draw_model(d)

### Maybe we don't need the second summary

In [ ]:
S2.remove()
draw_model(d)

### Generate some data

In [ ]:
d.generate(5).compute()

In [ ]:
S1.generate(5).compute()

### Rejection sampling

In [ ]:
# Specify the number of samples to sample and set up rejection sampling
n_samples = 10000
rej = Rejection(n_samples, d, [t1, t2], 10000)

In [ ]:
# Time and run the simulator in parallel
%time result = rej.infer(quantile=.01)

### Visualize results

In [ ]:
[t1_post, t2_post] = result['samples']
print("Number of accepted samples {} with threshold {:.2f}".format(len(t1_post), result['threshold']))
print("Posterior means: {:.2f} {:.2f}".format(t1_post.mean(), t2_post.mean()))

fig, ax = plt.subplots(ncols=2, figsize=(14,5));
ax[0].hist(t1_post, bins=20);
ax[0].set_title("Posterior for t1");
ax[1].hist(t2_post, bins=20);
ax[1].set_title("Posterior for t2");

### Maybe we do need the second summary

In [ ]:
S2 = Summary('S2', ac2, Y)
d.add_parent(S2)
d.reset()

In [ ]:
draw_model(d)

### Try if we improved

In [ ]:
%time result = rej.infer(quantile=.01)

### Print results

In [ ]:
[t1_post, t2_post] = result['samples']
print("Number of accepted samples {} with threshold {:.2f}".format(len(t1_post), result['threshold']))
print("Posterior means: {:.2f} {:.2f}".format(t1_post.mean(), t2_post.mean()))

fig, ax = plt.subplots(ncols=2, figsize=(14,5));
ax[0].hist(t1_post, bins=20);
ax[0].set_title("Posterior for t1");
ax[1].hist(t2_post, bins=20);
ax[1].set_title("Posterior for t2");

In [ ]:
# True values were
print(t1_0, "and ",t2_0)

## Parallelization

In [ ]:
# Visualize the underlying dask graph
dask.dot.dot_graph(d.generate(300, batch_size=100).dask)

### Cluster

In [ ]:
# Setup distributed
ip = None
client = Client(ip)
dask.set_options(get=client.get)

client

